In [63]:
import json
import os
import os.path
import re

from PIL import Image
import h5py
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import numpy as np

In [64]:
%run utils.ipynb
#from utils import *
%run config.ipynb
#from config import *

In [65]:
# category_dict = {'1':'car', '2':'airplane',...}
with open('categories_dict.json', 'r') as fd:
    category_dict = json.load(fd)

# Create dataset instance and Generate Dataloader

In [66]:
def get_loader(train=False, val=False, test=False):
    """ Returns a data loader for the desired split """
    split = ClassifySet(
        path_for(train=train, val=val, test=test, label=True),
        path_for(train=train, val=val, test=test, data=True),
    )
    loader = torch.utils.data.DataLoader(
        split,
        batch_size=batch_size,
        shuffle=train,  # only shuffle the data in training
        pin_memory=True,
        num_workers=data_workers,
    )
    return loader

In [67]:
class ClassifySet(data.Dataset):
    """ VQA dataset, open-ended """
    def __init__(self, label_path, data_path):
        super(ClassifySet, self).__init__()
        with open(label_path, 'r') as fd:
            self.label_json = json.load(fd)
        self.image_data = np.load(data_path)
        
    def __getitem__(self, item):
        l = int(self.label_json[item])
        image = self.image_data[item].reshape(28,28).astype(np.float32)
        return image, l

    def __len__(self):
        return len(self.label_json)